# Init

install and import necessary libraries

In order to run this you must first do two things
1. `pip install llama-cpp-python`
    * (if you get weird errors, consider switching to wsl)

2. install the model
    * `pip3 install huggingface-hub`
    
    * `huggingface-cli download TheBloke/CausalLM-7B-GGUF causallm_7b.Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False`


In [7]:
from llama_cpp import Llama
llm = Llama(
      model_path="sqlcoder2.Q4_K_M.gguf",
      # n_gpu_layers=-1, # Uncomment to use GPU acceleration
      # seed=1337, # Uncomment to set a specific seed
      n_ctx=2048, # Uncomment to increase the context window,
      
)

llama_model_loader: loaded meta data with 19 key-value pairs and 485 tensors from sqlcoder2.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = starcoder
llama_model_loader: - kv   1:                               general.name str              = StarCoder
llama_model_loader: - kv   2:                   starcoder.context_length u32              = 8192
llama_model_loader: - kv   3:                 starcoder.embedding_length u32              = 6144
llama_model_loader: - kv   4:              starcoder.feed_forward_length u32              = 24576
llama_model_loader: - kv   5:                      starcoder.block_count u32              = 40
llama_model_loader: - kv   6:             starcoder.attention.head_count u32              = 48
llama_model_loader: - kv   7:          starcoder.attention.head_count_kv u32              = 1


llama_model_loader: - type q5_K:   40 tensors
llama_model_loader: - type q6_K:   21 tensors
llm_load_vocab: mismatch in special tokens definition ( 19/49152 vs 0/49152 ).
llm_load_print_meta: format           = GGUF V2
llm_load_print_meta: arch             = starcoder
llm_load_print_meta: vocab type       = BPE
llm_load_print_meta: n_vocab          = 49152
llm_load_print_meta: n_merges         = 48891
llm_load_print_meta: n_ctx_train      = 8192
llm_load_print_meta: n_embd           = 6144
llm_load_print_meta: n_head           = 48
llm_load_print_meta: n_head_kv        = 1
llm_load_print_meta: n_layer          = 40
llm_load_print_meta: n_rot            = 128
llm_load_print_meta: n_embd_head_k    = 128
llm_load_print_meta: n_embd_head_v    = 128
llm_load_print_meta: n_gqa            = 48
llm_load_print_meta: n_embd_k_gqa     = 128
llm_load_print_meta: n_embd_v_gqa     = 128
llm_load_print_meta: f_norm_eps       = 1.0e-05
llm_load_print_meta: f_norm_rms_eps   = 0.0e+00
llm_load_print_met

LLAMA-CPP-Python API: https://llama-cpp-python.readthedocs.io/en/latest/api-reference/#high-level-api

In [9]:
llm('I\'m testing you!')['choices'][0]['text']
#if anything prints out that's not an error, it's good

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1783.46 ms
llama_print_timings:      sample time =       8.56 ms /    16 runs   (    0.54 ms per token,  1868.29 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =    5065.22 ms /    16 runs   (  316.58 ms per token,     3.16 tokens per second)
llama_print_timings:       total time =    5139.04 ms /    17 tokens


'\']\n\n        self.assertEqual(text_to_char_index("i\'m'

In [6]:
prompt = input('What is your question?')

print(llm("""## Task
Generate a SQL query to answer the following question:
`{prompt}`

### Database Schema
This query will run on a database whose schema is represented in this string:
CREATE TABLE products (
  product_id INTEGER PRIMARY KEY, -- Unique ID for each product
  name VARCHAR(50), -- Name of the product
  price DECIMAL(10,2), -- Price of each unit of the product
  quantity INTEGER  -- Current quantity in stock
);

CREATE TABLE sales (
  sale_id INTEGER PRIMARY KEY, -- Unique ID for each sale
  product_id INTEGER, -- ID of product sold
  customer_id INTEGER,  -- ID of customer who made purchase
  salesperson_id INTEGER, -- ID of salesperson who made the sale
  sale_date DATE, -- Date the sale occurred
  quantity INTEGER -- Quantity of product sold
);

-- sales.product_id can be joined with products.product_id

### SQL
Given the database schema, here is the SQL query that answers `{prompt}`:
```sql""", max_tokens=128))

Llama.generate: prefix-match hit

llama_print_timings:        load time =   42427.34 ms
llama_print_timings:      sample time =      10.13 ms /    16 runs   (    0.63 ms per token,  1579.78 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =    5085.72 ms /    16 runs   (  317.86 ms per token,     3.15 tokens per second)
llama_print_timings:       total time =    5171.91 ms /    17 tokens


{'id': 'cmpl-4c420e9c-7510-480f-a02b-a765a492ebed', 'object': 'text_completion', 'created': 1707572853, 'model': 'sqlcoder2.Q4_K_M.gguf', 'choices': [{'text': '\nSELECT SUM(sales.quantity) AS total_sales FROM sales;', 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 225, 'completion_tokens': 15, 'total_tokens': 240}}
